In [1]:
#sentence-transformers
#tf-keras 

In [2]:
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

df = pd.read_csv("Data/mens_clothing.csv")

df["ratings"] = pd.to_numeric(df["ratings"], errors="coerce")
df["no_of_ratings"] = df["no_of_ratings"].str.replace(",", "", regex=False)
df["no_of_ratings"] = pd.to_numeric(df["no_of_ratings"], errors="coerce").astype("Int64")

products = df.to_dict(orient="records")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13048\2733535531.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/mens_clothing.csv")


{'name': "Jockey 8820 Men's Super Combed Cotton Round Neck Sleeveless Vest with Extended Length for Easy Tuck (Pack of 3)", 'category': 'Innerwear', 'image': 'https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/81MfjxIbRsL._AC_UL320_.jpg', 'link': 'https://www.amazon.in/Jockey-Mens-Cotton-8901326034347_8820-0310-WHITE-White/dp/B00W04OTYM/ref=sr_1_5?qid=1679142473&s=apparel&sr=1-5', 'ratings': 4.2, 'no_of_ratings': 4240, 'discount_price_usd': 6.6, 'actual_price_usd': 7.88}


In [3]:
def build_text(product):
    return (
        f"Product: {product['name']}. "
        f"Category: {product['category']}. "
        f"Price: ${product['discount_price_usd']} (original ${product['actual_price_usd']}). "
        f"Rating: {product['ratings']} stars from {product['no_of_ratings']} reviews."
    )

Product: Jockey 8820 Men's Super Combed Cotton Round Neck Sleeveless Vest with Extended Length for Easy Tuck (Pack of 3). Category: Innerwear. Price: $6.6 (original $7.88). Rating: 4.2 stars from 4240 reviews.


In [4]:
product_texts = [build_text(prod) for prod in products]

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") #all-mpnet-base-v2

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13048\1078859149.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") #all-mpnet-base-v2
c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# vectorstore = FAISS.from_texts(product_texts, embeddings)
# vectorstore.save_local("faiss_product_index")

In [8]:
vectorstore = FAISS.load_local(
    "faiss_product_index",
    embeddings,
    allow_dangerous_deserialization=True
)

In [9]:
len(vectorstore.docstore._dict)

73778

In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
import requests
import pandas as pd
import os

os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

llm = ChatOpenAI(
    model="openai/gpt-4.1-mini", 
    temperature=0
)

In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [84]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  
)

from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=False,
    allow_dangerous_code=True
)


In [96]:
def classify_query(query):
    prompt = f"""
    You are a classifier for a shopping assistant.
    Classify the user question into one of two categories:
    - "RAG": If the question is about **searching, reading, or retrieving textual information** related to products or Q&A.
    - "AGENT": If the question requires **filtering, calculation, aggregation, comparison, or price-based queries** about products, such as cheapest, most expensive, average price, counting items, filtering by size/color/brand, or comparing multiple products.
    Only return one of these two words: RAG or AGENT.
    Question: {query}
    """
    result = llm.invoke(prompt)
    return result.content.strip()
    
def answer_query(query):
    intent = classify_query(query)

    if intent == "AGENT":
        print("[Agent Mode]")
        return agent.run(query)
    else:
        print("[RAG Mode]")
        # print("\n----------- Source documents -----------")
        # for doc in result["source_documents"]:
        #     print("-", doc.page_content)
        return qa_chain({"query": query})["result"]

In [99]:
query = "Give me some products of NEVA"

In [100]:
print(answer_query(query))

[RAG Mode]
Here are some NEVA products:

1. NEVA Men Lower Thermal Esancia - Milange Grey, Price: $6.9
2. NEVA Men Pure Cotton Solid Trunk Brief, Price: $2.58
3. NEVA Men Lower Thermal Esancia - Off White, Price: $6.9
4. Neva Men's Off White Cotton Thermal, Price: $6.58, Rating: 4.3 stars
5. Neva Upper Thermal for Men (OMS1-Grey), Price: $6.0, Rating: 4.6 stars
6. NEVA Gents Thermal Set with Fur Inside for Men (Glaccia) - Anthra, Price: $22.66, Rating: 5.0 stars
7. NEVA Men Full Sleeve Upper Thermal Esancia - Anthra, Price: $6.68
8. NEVA Premium LADIESTHERMAL Set Upper and Lower Antra (Set), Price: $11.98
9. Neva Upper Thermal for Men (OMS1-Grey, 110 cm), Price: $6.0, Rating: 5.0 stars
10. NEVA Premium Inner Thermal Set of Upper and Lower with Free 1 Pair of Socks, Price: $7.19, Rating: 4.2 stars

Let me know if you want details on any specific product!
